# Trading Volume Decomposition: Signal Tier x Trade Classification

This notebook decomposes execution volume by **signal strength** and **trade classification** using:
- `{TEAM_ID}/targets.csv` for canonical classification (`NEW_ENTRY`, `RESIZE`, `FLIP`, `EXIT`)
- `{TEAM_ID}/order_events.csv` for order lifecycle and fills
- `{TEAM_ID}/signals.csv` for rebalance signal magnitude
- `{TEAM_ID}/daily_snapshots.csv` for NAV normalization

Charts included:
1. Filled Notional Heatmap: Signal Tier x Classification
2. Order Count & Fill Rate by Tier x Side
3. Daily Volume Profile by Signal Tier over Scale Days + theoretical schedules
4. Turnover Decomposition Waterfall (stacked) by Classification
5. Signal Magnitude vs Filled Notional Scatter + regression

In [ ]:
import re
import warnings
from io import StringIO

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

from QuantConnect import *
from QuantConnect.Research import QuantBook
from config import TEAM_ID

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 180)

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

warnings.filterwarnings('ignore', category=FutureWarning)


In [ ]:
qb = QuantBook()
print('QuantBook initialized')


def read_csv_from_store(key):
    try:
        if not qb.ObjectStore.ContainsKey(key):
            print(f'ObjectStore key not found: {key}')
            return None
        content = qb.ObjectStore.Read(key)
        if not content:
            print(f'Empty ObjectStore key: {key}')
            return None
        return pd.read_csv(StringIO(content))
    except Exception as e:
        print(f'Error reading {key}: {e}')
        return None


def to_numeric(df, cols):
    for col in cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    return df


def parse_tag_value(tag, key):
    if pd.isna(tag):
        return np.nan
    match = re.search(rf'{key}=([^;]+)', str(tag))
    return match.group(1).strip() if match else np.nan


def tier_from_strength(x):
    if pd.isna(x):
        return 'unknown'
    x = abs(float(x))
    if x >= 0.70:
        return 'strong'
    if x >= 0.30:
        return 'moderate'
    return 'weak'


def normalize_side(direction, quantity):
    d = str(direction).strip().lower()
    if d in ('buy', 'sell'):
        return d
    q = pd.to_numeric(quantity, errors='coerce')
    if pd.isna(q):
        return 'unknown'
    return 'buy' if q > 0 else ('sell' if q < 0 else 'unknown')


def choose_first_valid(series, default=np.nan):
    for x in series:
        if pd.notna(x) and str(x).strip() != '':
            return x
    return default


def choose_tier(series):
    valid = [str(x).strip().lower() for x in series if pd.notna(x) and str(x).strip() != '']
    for tier in ('strong', 'moderate', 'weak', 'exit'):
        if tier in valid:
            return tier
    return 'unknown'


def choose_classification(series):
    valid = [str(x).strip().upper() for x in series if pd.notna(x) and str(x).strip() != '']
    for c in ('NEW_ENTRY', 'RESIZE', 'FLIP', 'EXIT', 'HOLD'):
        if c in valid:
            return c
    return 'UNKNOWN'


def build_scaling_schedule(scaling_days, front_load_factor):
    n = max(1, int(scaling_days))
    if n <= 1:
        return [1.0]
    exponent = 1.0 / float(front_load_factor)
    sched = [round((i / n) ** exponent, 4) for i in range(1, n + 1)]
    sched[-1] = 1.0
    return sched


In [ ]:
# Load ObjectStore datasets

df_snap = read_csv_from_store(f'{TEAM_ID}/daily_snapshots.csv')
df_targets = read_csv_from_store(f'{TEAM_ID}/targets.csv')
df_events = read_csv_from_store(f'{TEAM_ID}/order_events.csv')
df_signals = read_csv_from_store(f'{TEAM_ID}/signals.csv')

if df_snap is None or df_targets is None or df_events is None:
    raise ValueError('daily_snapshots.csv, targets.csv, and order_events.csv are required.')

# Snapshots -> NAV by date
snap = df_snap.copy()
snap['date'] = pd.to_datetime(snap['date']).dt.normalize()
snap = to_numeric(snap, ['nav'])
nav_by_date = snap[['date', 'nav']].drop_duplicates(['date'])

# Targets -> canonical week/symbol classification and scale-day map
tgt = df_targets.copy()
tgt['date'] = pd.to_datetime(tgt['date']).dt.normalize()
num_cols = ['start_w', 'weekly_target_w', 'scheduled_fraction', 'scheduled_w', 'actual_w', 'scale_day']
tgt = to_numeric(tgt, num_cols)
tgt['scale_day'] = tgt['scale_day'].fillna(0).astype(int)
tgt['week_id'] = tgt['week_id'].astype(str)
invalid_week = tgt['week_id'].isin(['', 'nan', 'None'])
tgt.loc[invalid_week, 'week_id'] = tgt.loc[invalid_week, 'date'].dt.strftime('%Y-%m-%d')
tgt['classification'] = tgt['classification'].astype(str).str.upper().str.strip().replace({'': 'UNKNOWN'})

week_symbol = (
    tgt.sort_values(['week_id', 'symbol', 'date'])
       .drop_duplicates(['week_id', 'symbol'], keep='first')
       [['week_id', 'symbol', 'date', 'classification', 'start_w', 'weekly_target_w']]
       .rename(columns={'date': 'week_start_date'})
)

# For scale-day join on fills
target_daily_map = (
    tgt[['date', 'symbol', 'week_id', 'scale_day', 'classification']]
      .sort_values(['date', 'symbol'])
      .drop_duplicates(['date', 'symbol'], keep='last')
      .rename(columns={
          'week_id': 'week_id_target',
          'scale_day': 'scale_day_target',
          'classification': 'classification_target'
      })
)

# Signals -> week/symbol strength + tier
if df_signals is not None and len(df_signals):
    sig = df_signals.copy()
    sig['date'] = pd.to_datetime(sig['date']).dt.normalize()
    sig = to_numeric(sig, ['magnitude'])
    sig['week_id'] = sig['date'].dt.strftime('%Y-%m-%d')
    sig['signal_strength_signal'] = sig['magnitude'].abs()
    sig['signal_tier_signal'] = sig['signal_strength_signal'].apply(tier_from_strength)
    sig_week = (
        sig.sort_values(['week_id', 'symbol', 'date'])
           .drop_duplicates(['week_id', 'symbol'], keep='last')
           [['week_id', 'symbol', 'signal_strength_signal', 'signal_tier_signal']]
    )
else:
    sig_week = pd.DataFrame(columns=['week_id', 'symbol', 'signal_strength_signal', 'signal_tier_signal'])

# Event-tag fallback strength/tier at week/symbol level
evt_tmp = df_events.copy()
evt_tmp['date'] = pd.to_datetime(evt_tmp['date']).dt.normalize()
evt_tmp['week_id_tag'] = evt_tmp['tag'].apply(lambda t: parse_tag_value(t, 'week_id'))
evt_tmp['tier_tag'] = evt_tmp['tag'].apply(lambda t: parse_tag_value(t, 'tier'))
evt_tmp['signal_tag'] = pd.to_numeric(evt_tmp['tag'].apply(lambda t: parse_tag_value(t, 'signal')), errors='coerce').abs()
evt_tmp['week_id_tag'] = evt_tmp['week_id_tag'].fillna(evt_tmp['date'].dt.strftime('%Y-%m-%d'))

tag_week = (
    evt_tmp.groupby(['week_id_tag', 'symbol'], as_index=False)
           .agg(
               signal_strength_tag=('signal_tag', 'max'),
               signal_tier_tag=('tier_tag', choose_tier)
           )
           .rename(columns={'week_id_tag': 'week_id'})
)

# Build week/symbol lookup with robust signal tier assignment
week_symbol = week_symbol.merge(sig_week, on=['week_id', 'symbol'], how='left')
week_symbol = week_symbol.merge(tag_week, on=['week_id', 'symbol'], how='left')
week_symbol['signal_strength'] = week_symbol['signal_strength_signal'].fillna(week_symbol['signal_strength_tag'])
week_symbol['signal_tier'] = week_symbol['signal_tier_signal'].fillna(week_symbol['signal_tier_tag'])
week_symbol['signal_tier'] = week_symbol['signal_tier'].fillna('unknown').str.lower().str.strip()

# EXIT weeks often have no current signal; attribute to prior known tier per symbol.
week_symbol = week_symbol.sort_values(['symbol', 'week_start_date'])
week_symbol['prior_known_tier'] = (
    week_symbol['signal_tier']
      .where(week_symbol['signal_tier'].isin(['strong', 'moderate', 'weak']))
      .groupby(week_symbol['symbol'])
      .ffill()
)
mask_exit_missing = (week_symbol['classification'] == 'EXIT') & (~week_symbol['signal_tier'].isin(['strong', 'moderate', 'weak']))
week_symbol.loc[mask_exit_missing, 'signal_tier'] = week_symbol.loc[mask_exit_missing, 'prior_known_tier']
week_symbol['signal_tier'] = week_symbol['signal_tier'].fillna('unknown')

print(f"Rows -> snapshots: {len(snap):,}, targets: {len(tgt):,}, events: {len(df_events):,}, signals: {0 if df_signals is None else len(df_signals):,}")
print(f"Week-symbol rows: {len(week_symbol):,}")
display(week_symbol.head())


In [ ]:
# Enrich order events and fill rows with week/classification/tier and NAV-normalized notional

events = df_events.copy()
events['date'] = pd.to_datetime(events['date']).dt.normalize()
for c in ['quantity', 'fill_quantity', 'fill_price']:
    if c in events.columns:
        events[c] = pd.to_numeric(events[c], errors='coerce')

events['week_id_tag'] = events['tag'].apply(lambda t: parse_tag_value(t, 'week_id'))
events['tier_tag'] = events['tag'].apply(lambda t: parse_tag_value(t, 'tier'))
events['signal_tag'] = pd.to_numeric(events['tag'].apply(lambda t: parse_tag_value(t, 'signal')), errors='coerce').abs()
events['side'] = [normalize_side(d, q) for d, q in zip(events.get('direction', pd.Series(index=events.index)), events.get('quantity', pd.Series(index=events.index)))]

# Attach target day metadata (scale_day + fallback week/classification)
events = events.merge(target_daily_map, on=['date', 'symbol'], how='left')
events['week_id'] = events['week_id_tag']
missing_week = events['week_id'].isna() | events['week_id'].astype(str).str.strip().eq('')
events.loc[missing_week, 'week_id'] = events.loc[missing_week, 'week_id_target']
events['week_id'] = events['week_id'].fillna(events['date'].dt.strftime('%Y-%m-%d')).astype(str)

# Week-level classification + signal fields
events = events.merge(
    week_symbol[['week_id', 'symbol', 'classification', 'signal_tier', 'signal_strength', 'week_start_date']],
    on=['week_id', 'symbol'],
    how='left'
)

events['classification'] = events['classification'].fillna(events['classification_target'])
events['classification'] = events['classification'].fillna('UNKNOWN').astype(str).str.upper().str.strip()

# Fallback tier from tag, then from strength
events['signal_tier'] = events['signal_tier'].fillna(events['tier_tag'])
events['signal_strength'] = events['signal_strength'].fillna(events['signal_tag'])
events['signal_tier'] = events['signal_tier'].fillna(events['signal_strength'].apply(tier_from_strength))
events['signal_tier'] = events['signal_tier'].fillna('unknown').astype(str).str.lower().str.strip()

# Derive fill metrics
fills = events.copy()
fills['fill_quantity_abs'] = fills['fill_quantity'].abs().fillna(0.0)
fills['fill_price_abs'] = fills['fill_price'].abs().fillna(0.0)
fills['fill_notional'] = fills['fill_quantity_abs'] * fills['fill_price_abs']
fills = fills[fills['fill_notional'] > 0].copy()

fills = fills.merge(nav_by_date, on='date', how='left')
fills['fill_pct_nav'] = np.where(fills['nav'] > 1e-12, fills['fill_notional'] / fills['nav'], np.nan)

fills['scale_day'] = fills['scale_day_target']
fills['scale_day'] = pd.to_numeric(fills['scale_day'], errors='coerce')

print(f'Fill rows: {len(fills):,}')
print(f'Unique order_ids: {events["order_id"].nunique():,}')
display(fills[['date', 'week_id', 'symbol', 'signal_tier', 'classification', 'side', 'fill_notional', 'fill_pct_nav', 'scale_day']].head())


## 1) Filled Notional Heatmap: Signal Tier x Classification

Cell color is **total filled notional as % NAV** aggregated over the full sample.

In [ ]:
tier_rows = ['strong', 'moderate', 'weak']
class_cols = ['NEW_ENTRY', 'RESIZE', 'FLIP', 'EXIT']

heat = (
    fills[fills['signal_tier'].isin(tier_rows) & fills['classification'].isin(class_cols)]
      .groupby(['signal_tier', 'classification'], as_index=False)
      .agg(total_fill_pct_nav=('fill_pct_nav', 'sum'))
)

heat_pivot = (
    heat.pivot(index='signal_tier', columns='classification', values='total_fill_pct_nav')
        .reindex(index=tier_rows, columns=class_cols)
        .fillna(0.0)
)

plt.figure(figsize=(10, 5))
sns.heatmap(
    100 * heat_pivot,
    annot=True,
    fmt='.2f',
    cmap='YlGnBu',
    cbar_kws={'label': 'Total Filled Notional (% NAV)'}
)
plt.title('Filled Notional Heatmap: Signal Tier x Classification')
plt.xlabel('Trade Classification')
plt.ylabel('Signal Tier')
plt.tight_layout()
plt.show()

display((100 * heat_pivot).round(3))


## 2) Order Count & Fill Rate by Tier x Side

`fill_rate` below is the share of orders with **any executed quantity** (`abs(fill_quantity) > 0`) across their lifecycle.

In [ ]:
# Build order-level summary from lifecycle events
order_events = events.copy().reset_index(drop=False).rename(columns={'index': 'event_idx'})
order_events = order_events.sort_values(['order_id', 'event_idx'])

order_summary = (
    order_events.groupby('order_id', as_index=False)
               .agg(
                   side=('side', lambda s: choose_first_valid(s, default='unknown')),
                   signal_tier=('signal_tier', choose_tier),
                   classification=('classification', choose_classification),
                   final_status=('status', lambda s: choose_first_valid(list(s)[::-1], default='unknown')),
                   any_fill=('fill_quantity', lambda s: float((pd.to_numeric(s, errors='coerce').abs().fillna(0.0) > 0).any()))
               )
)

plot_tiers = ['strong', 'moderate', 'weak', 'exit']
plot_sides = ['buy', 'sell']

metric = (
    order_summary[order_summary['signal_tier'].isin(plot_tiers) & order_summary['side'].isin(plot_sides)]
      .groupby(['side', 'signal_tier'], as_index=False)
      .agg(
          order_count=('order_id', 'count'),
          fill_rate_pct=('any_fill', lambda x: 100.0 * np.mean(x))
      )
)

palette = {
    'strong': '#1f77b4',
    'moderate': '#ff7f0e',
    'weak': '#2ca02c',
    'exit': '#9467bd'
}

fig, axes = plt.subplots(2, 2, figsize=(14, 10), sharex=True)
for j, side in enumerate(plot_sides):
    sub = metric[metric['side'] == side].copy()
    sub['signal_tier'] = pd.Categorical(sub['signal_tier'], categories=plot_tiers, ordered=True)
    sub = sub.sort_values('signal_tier')

    sns.barplot(
        data=sub,
        x='signal_tier',
        y='order_count',
        order=plot_tiers,
        palette=palette,
        ax=axes[0, j]
    )
    axes[0, j].set_title(f'Order Count ({side.title()})')
    axes[0, j].set_xlabel('Signal Tier')
    axes[0, j].set_ylabel('Orders')
    axes[0, j].grid(axis='y', alpha=0.3)

    sns.barplot(
        data=sub,
        x='signal_tier',
        y='fill_rate_pct',
        order=plot_tiers,
        palette=palette,
        ax=axes[1, j]
    )
    axes[1, j].set_title(f'Fill Rate ({side.title()})')
    axes[1, j].set_xlabel('Signal Tier')
    axes[1, j].set_ylabel('Fill Rate (%)')
    axes[1, j].set_ylim(0, 100)
    axes[1, j].grid(axis='y', alpha=0.3)

plt.suptitle('Order Count & Fill Rate by Signal Tier x Side', y=1.02)
plt.tight_layout()
plt.show()

display(metric.sort_values(['side', 'signal_tier']))


## 3) Daily Volume Profile by Signal Tier (Scale Day 0-4)

Stacked area shows actual filled notional (% NAV) by scale day and tier.
Dashed overlays show each tier's theoretical daily profile from front-load factors:
- strong: `2.0`
- moderate: `1.3`
- weak: `1.0`

In [ ]:
scale_days = [0, 1, 2, 3, 4]
tiers = ['strong', 'moderate', 'weak']

profile = fills.copy()
profile = profile[
    profile['signal_tier'].isin(tiers) &
    profile['scale_day'].notna() &
    (profile['scale_day'] >= 0) &
    (profile['scale_day'] <= 4)
].copy()
profile['scale_day'] = profile['scale_day'].astype(int)

actual = (
    profile.groupby(['scale_day', 'signal_tier'], as_index=False)
           .agg(fill_pct_nav=('fill_pct_nav', 'sum'))
)
actual_pivot = (
    actual.pivot(index='scale_day', columns='signal_tier', values='fill_pct_nav')
          .reindex(index=scale_days, columns=tiers)
          .fillna(0.0)
)

x = np.array(scale_days)
stack_values = [100 * actual_pivot[t].to_numpy() for t in tiers]
colors = {'strong': '#1f77b4', 'moderate': '#ff7f0e', 'weak': '#2ca02c'}

fig, ax = plt.subplots(figsize=(14, 7))
ax.stackplot(
    x,
    stack_values,
    labels=[f'{t} actual' for t in tiers],
    colors=[colors[t] for t in tiers],
    alpha=0.65
)

# Overlay theoretical daily schedule, scaled by each tier's total realized notional.
schedule_cfg = {'strong': 2.0, 'moderate': 1.3, 'weak': 1.0}
for tier in tiers:
    total = float(actual_pivot[tier].sum())
    if total <= 0:
        continue
    cumulative = build_scaling_schedule(5, schedule_cfg[tier])
    increments = np.diff([0.0] + cumulative)
    expected_daily = 100 * total * increments
    ax.plot(
        x,
        expected_daily,
        linestyle='--',
        linewidth=2,
        color=colors[tier],
        label=f'{tier} theoretical'
    )

ax.set_title('Daily Volume Profile by Signal Tier over Scale Days')
ax.set_xlabel('Scale Day')
ax.set_ylabel('Filled Notional (% NAV)')
ax.set_xticks(scale_days)
ax.grid(axis='y', alpha=0.3)
ax.legend(ncol=2)
plt.tight_layout()
plt.show()

display((100 * actual_pivot).round(4))


## 4) Turnover Decomposition Waterfall by Classification

Stacked bars by rebalance week show total filled turnover (% NAV) decomposed into:
`NEW_ENTRY`, `RESIZE`, `FLIP`, `EXIT`.

In [ ]:
class_order = ['NEW_ENTRY', 'RESIZE', 'FLIP', 'EXIT']

week_class = (
    fills[fills['classification'].isin(class_order)]
      .groupby(['week_id', 'classification'], as_index=False)
      .agg(fill_pct_nav=('fill_pct_nav', 'sum'))
)

week_dates = (
    week_symbol[['week_id', 'week_start_date']]
      .drop_duplicates('week_id')
)
week_class = week_class.merge(week_dates, on='week_id', how='left')
week_class['week_start_date'] = pd.to_datetime(week_class['week_start_date'])

pivot = (
    week_class.pivot_table(
        index='week_start_date',
        columns='classification',
        values='fill_pct_nav',
        aggfunc='sum',
        fill_value=0.0
    )
    .reindex(columns=class_order)
    .sort_index()
)

fig, ax = plt.subplots(figsize=(16, 7))
bottom = np.zeros(len(pivot))
x = np.arange(len(pivot))
color_map = {
    'NEW_ENTRY': '#1f77b4',
    'RESIZE': '#ff7f0e',
    'FLIP': '#2ca02c',
    'EXIT': '#d62728'
}

for cls in class_order:
    vals = 100 * pivot[cls].to_numpy()
    ax.bar(x, vals, bottom=bottom, label=cls, color=color_map[cls], width=0.8)
    bottom += vals

labels = [d.strftime('%Y-%m-%d') for d in pivot.index]
ax.set_xticks(x)
ax.set_xticklabels(labels, rotation=45, ha='right')
ax.set_ylabel('Filled Turnover (% NAV)')
ax.set_xlabel('Rebalance Week Start')
ax.set_title('Turnover Decomposition by Classification (Waterfall-Style Stacked Bars)')
ax.grid(axis='y', alpha=0.3)
ax.legend(title='Classification')
plt.tight_layout()
plt.show()

display((100 * pivot).round(4).tail(20))


## 5) Signal Magnitude vs Filled Notional (Symbol-Week)

Each point is one symbol-week. X = `|signal magnitude|`, Y = total filled notional (% NAV), color = classification.
Black line is an overall OLS trend line.

In [ ]:
class_order = ['NEW_ENTRY', 'RESIZE', 'FLIP', 'EXIT']

symbol_week = (
    fills[fills['classification'].isin(class_order)]
      .groupby(['week_id', 'symbol', 'classification'], as_index=False)
      .agg(filled_pct_nav=('fill_pct_nav', 'sum'))
)

symbol_week = symbol_week.merge(
    week_symbol[['week_id', 'symbol', 'signal_strength']],
    on=['week_id', 'symbol'],
    how='left'
)

symbol_week['signal_strength'] = pd.to_numeric(symbol_week['signal_strength'], errors='coerce')
symbol_week = symbol_week[symbol_week['signal_strength'].notna() & (symbol_week['signal_strength'] > 0)].copy()
symbol_week['filled_pct_nav_pct'] = 100 * symbol_week['filled_pct_nav']

plt.figure(figsize=(12, 7))

sns.scatterplot(
    data=symbol_week,
    x='signal_strength',
    y='filled_pct_nav_pct',
    hue='classification',
    hue_order=class_order,
    palette={'NEW_ENTRY': '#1f77b4', 'RESIZE': '#ff7f0e', 'FLIP': '#2ca02c', 'EXIT': '#d62728'},
    alpha=0.75,
    s=55
)

if len(symbol_week) >= 2:
    sns.regplot(
        data=symbol_week,
        x='signal_strength',
        y='filled_pct_nav_pct',
        scatter=False,
        color='black',
        line_kws={'linewidth': 2, 'alpha': 0.9}
    )

plt.title('Signal Magnitude vs Filled Notional (% NAV) by Classification')
plt.xlabel('|Signal Magnitude|')
plt.ylabel('Filled Notional (% NAV) per Symbol-Week')
plt.grid(alpha=0.3)
plt.legend(title='Classification')
plt.tight_layout()
plt.show()

corr = symbol_week[['signal_strength', 'filled_pct_nav_pct']].corr().iloc[0, 1] if len(symbol_week) >= 2 else np.nan
print(f'Pearson correlation(|signal|, filled %NAV): {corr:.4f}' if pd.notna(corr) else 'Not enough points for correlation.')
display(symbol_week.sort_values('filled_pct_nav', ascending=False).head(20))


## Auto Summary: Top Trading Drivers

This section prints compact diagnostics from the computed tables above so you can quickly see where execution is concentrated.


In [ ]:
# Auto summary of key drivers from the decomposition analysis

if 'fills' not in globals() or len(fills) == 0:
    print('No fill data available for summary.')
else:
    class_order = ['NEW_ENTRY', 'RESIZE', 'FLIP', 'EXIT']
    tier_order = ['strong', 'moderate', 'weak', 'exit', 'unknown']

    # 1) Top tier x classification buckets by total filled % NAV
    bucket = (
        fills.groupby(['signal_tier', 'classification'], as_index=False)
             .agg(filled_pct_nav=('fill_pct_nav', 'sum'))
    )
    bucket = bucket.sort_values('filled_pct_nav', ascending=False)
    total_pct_nav = bucket['filled_pct_nav'].sum()
    bucket['share_pct'] = np.where(total_pct_nav > 1e-12, 100.0 * bucket['filled_pct_nav'] / total_pct_nav, np.nan)

    print('Top Tier x Classification Buckets (by filled %NAV):')
    display(bucket.head(12))

    # 2) Top rebalance weeks by turnover and their composition
    week_mix = (
        fills[fills['classification'].isin(class_order)]
            .groupby(['week_id', 'classification'], as_index=False)
            .agg(filled_pct_nav=('fill_pct_nav', 'sum'))
    )
    week_pivot = (
        week_mix.pivot_table(index='week_id', columns='classification', values='filled_pct_nav', aggfunc='sum', fill_value=0.0)
                .reindex(columns=class_order, fill_value=0.0)
                .reset_index()
    )
    week_pivot['turnover_pct_nav'] = week_pivot[class_order].sum(axis=1)
    for c in class_order:
        week_pivot[f'{c}_share_pct'] = np.where(
            week_pivot['turnover_pct_nav'] > 1e-12,
            100.0 * week_pivot[c] / week_pivot['turnover_pct_nav'],
            np.nan
        )

    top_weeks = week_pivot.sort_values('turnover_pct_nav', ascending=False).head(10)
    print('Top Rebalance Weeks by Filled Turnover (%NAV):')
    display(top_weeks[['week_id', 'turnover_pct_nav'] + class_order + [f'{c}_share_pct' for c in class_order]])

    # 3) Fill-rate asymmetry by side and tier (order-level)
    if 'order_summary' in globals() and len(order_summary):
        asym = (
            order_summary[order_summary['side'].isin(['buy', 'sell'])]
                .groupby(['signal_tier', 'side'], as_index=False)
                .agg(
                    orders=('order_id', 'count'),
                    fill_rate=('any_fill', 'mean')
                )
        )
        asym['fill_rate_pct'] = 100.0 * asym['fill_rate']
        asym = asym.sort_values(['signal_tier', 'side'])

        asym_pivot = asym.pivot(index='signal_tier', columns='side', values='fill_rate_pct')
        asym_pivot['buy_minus_sell_fill_rate_pct'] = asym_pivot.get('buy', np.nan) - asym_pivot.get('sell', np.nan)

        print('Fill-Rate by Tier x Side (%):')
        display(asym)
        print('Buy - Sell Fill-Rate Spread (% points):')
        display(asym_pivot[['buy_minus_sell_fill_rate_pct']].sort_values('buy_minus_sell_fill_rate_pct', ascending=False))

    # 4) Signal strength vs filled notional relationship
    if 'symbol_week' in globals() and len(symbol_week) >= 2:
        corr = symbol_week[['signal_strength', 'filled_pct_nav']].corr().iloc[0, 1]
        slope, intercept = np.polyfit(symbol_week['signal_strength'], symbol_week['filled_pct_nav'], 1)
        print(
            f"Signal-strength linkage: corr={corr:.4f}, "
            f"slope={slope:.6f} (filled %NAV per 1.0 signal-magnitude unit), "
            f"intercept={intercept:.6f}"
        )
